Week 5 : Data Modelling, storage, filtering and Sampling

Dataset : NYC Taxi dataset [link text](https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-08.parquet)


Yellow taxi trip data for august 2025


In [9]:
import pyarrow
pyarrow.__doc__ #tells us what the module does

'\nPyArrow is the python implementation of Apache Arrow.\n\nApache Arrow is a cross-language development platform for in-memory data.\nIt specifies a standardized language-independent columnar memory format for\nflat and hierarchical data, organized for efficient analytic operations on\nmodern hardware. It also provides computational libraries and zero-copy\nstreaming messaging and interprocess communication.\n\nFor more information see the official page at https://arrow.apache.org\n'

In [10]:
pyarrow.__version__

'18.1.0'

In [11]:
import pyarrow.parquet as pq
trips = pq.read_table('/content/drive/MyDrive/Data Engineering/Tutorial/week 5/data/yellow_tripdata_2025-08.parquet')



In [12]:
type(trips)

pyarrow.lib.Table

In [13]:
trips.shape

(3574091, 20)

In [14]:
trips = trips.to_pandas()

In [15]:
type(trips)

pandas.core.frame.DataFrame

In [16]:
#investigate the header
trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,2,2025-08-01 00:52:23,2025-08-01 01:12:20,1.0,8.44,1.0,N,138,141,1,33.8,6.0,0.5,5.00,6.94,1.0,57.49,2.5,1.75,0.00
1,2,2025-08-01 00:03:01,2025-08-01 00:15:33,2.0,4.98,1.0,N,138,193,1,21.2,6.0,0.5,0.00,0.00,1.0,30.45,0.0,1.75,0.00
2,7,2025-08-01 00:24:38,2025-08-01 00:24:38,2.0,1.89,1.0,N,249,45,1,14.2,0.0,0.5,3.99,0.00,1.0,23.94,2.5,0.00,0.75
3,7,2025-08-01 00:48:19,2025-08-01 00:48:19,1.0,2.35,1.0,N,79,229,1,11.4,0.0,0.5,3.43,0.00,1.0,20.58,2.5,0.00,0.75
4,2,2025-08-01 00:25:34,2025-08-01 00:33:18,1.0,2.14,1.0,N,43,48,1,11.4,1.0,0.5,2.57,0.00,1.0,19.72,2.5,0.00,0.75


In [18]:
#create a copy of the data as df
df=trips.copy()

In [21]:
df['trip id'] = df.index

In [22]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee',
       'cbd_congestion_fee', 'trip id'],
      dtype='object')

In [23]:
#create datetime dimension
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday
datetime_dim['datetime_id'] = datetime_dim.index

# datetime_dim = datetime_dim.rename(columns={'tpep_pickup_datetime': 'datetime_id'}).reset_index(drop=True)
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday','tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]
#
datetime_dim.head()


,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2025-08-01 00:52:23,0,1,8,2025,4,2025-08-01 01:12:20,1,1,8,2025,4
1,1,2025-08-01 00:03:01,0,1,8,2025,4,2025-08-01 00:15:33,0,1,8,2025,4
2,2,2025-08-01 00:24:38,0,1,8,2025,4,2025-08-01 00:24:38,0,1,8,2025,4
3,3,2025-08-01 00:48:19,0,1,8,2025,4,2025-08-01 00:48:19,0,1,8,2025,4
4,4,2025-08-01 00:25:34,0,1,8,2025,4,2025-08-01 00:33:18,0,1,8,2025,4


In [25]:
#create passenger count dimension
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]
passenger_count_dim.head()#view first 5 rows

,passenger_count_id,passenger_count
0,0,1.0
1,1,2.0
2,2,2.0
3,3,1.0
4,4,1.0


Vendor Dimension:


,vendor_id,VendorID
0,0,2
1,1,2
2,2,7
3,3,7
4,4,2


In [26]:
#create trip distance dimensionand view first 5 rows
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]
trip_distance_dim.head()


,trip_distance_id,trip_distance
0,0,8.44
1,1,4.98
2,2,1.89
3,3,2.35
4,4,2.14


Location Dimension:


,location_id,PULocationID,DOLocationID
0,0,138,141
1,1,138,193
2,2,249,45
3,3,79,229
4,4,43,48


In [45]:
#create rate code dimension and view first 5 rows
rate_code_type = {
1:"Standard rate",
2:"JFK",
3:"Newark",
4:"Nassau or Westchester",
5:"Negotiated fare",
6:"Group ride"
}
rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]
rate_code_dim.head()

,rate_code_id,RatecodeID,rate_code_name
0,0,1.0,Standard rate
1,1,1.0,Standard rate
2,2,1.0,Standard rate
3,3,1.0,Standard rate
4,4,1.0,Standard rate


In [46]:
#create payment type dimension and view first 5 rows
payment_type_name = {
1:"Credit card",
2:"Cash",
3:"No charge",
4:"Dispute",
5:"Unknown",
6:"Voided trip"
}
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]
payment_type_dim.head()

,payment_type_id,payment_type,payment_type_name
0,0,1,Credit card
1,1,1,Credit card
2,2,1,Credit card
3,3,1,Credit card
4,4,1,Credit card


In [50]:
#createa fact table
fact_table = df.merge(passenger_count_dim, left_on='trip id', right_on='passenger_count_id') \
.merge(trip_distance_dim, left_on='trip id', right_on='trip_distance_id') \
.merge(rate_code_dim, left_on='trip id', right_on='rate_code_id') \
.merge(datetime_dim, left_on='trip id', right_on='datetime_id') \
.merge(payment_type_dim, left_on='trip id', right_on='payment_type_id') \
.merge(vendor_dim, left_on='trip id', right_on='vendor_id') \
.merge(location_dim, left_on='trip id', right_on='location_id') \
[['trip id', 'datetime_id', 'passenger_count_id',
'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag',
'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
'tolls_amount',
'improvement_surcharge', 'total_amount', 'vendor_id', 'location_id']]

In [51]:
#investigate the fact table
fact_table.head()

,trip id,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,vendor_id,location_id
0,0,0,0,0,0,N,0,33.8,6.0,0.5,5.00,6.94,1.0,57.49,0,0
1,1,1,1,1,1,N,1,21.2,6.0,0.5,0.00,0.00,1.0,30.45,1,1
2,2,2,2,2,2,N,2,14.2,0.0,0.5,3.99,0.00,1.0,23.94,2,2
3,3,3,3,3,3,N,3,11.4,0.0,0.5,3.43,0.00,1.0,20.58,3,3
4,4,4,4,4,4,N,4,11.4,1.0,0.5,2.57,0.00,1.0,19.72,4,4


Task : Load the dimension and fact tables into a SQLite database

In [52]:
#Create a connection to the SQLite database.
import sqlite3
conn = sqlite3.connect('nyc_taxi_data.db')

Load each of the dimension DataFrames (`datetime_dim`, `passenger_count_dim`,
`trip_distance_dim`, `rate_code_dim`, `payment_type_dim`) into separate tables in the
SQLite database.

In [53]:
#Write each dimension DataFrame to a separate table in the SQLite database

In [54]:
datetime_dim.to_sql('datetime_dim', conn, if_exists='replace', index=False)
passenger_count_dim.to_sql('passenger_count_dim', conn, if_exists='replace',
index=False)
trip_distance_dim.to_sql('trip_distance_dim', conn, if_exists='replace',
index=False)
rate_code_dim.to_sql('rate_code_dim', conn, if_exists='replace', index=False)
payment_type_dim.to_sql('payment_type_dim', conn, if_exists='replace', index=False)

3574091

In [55]:
#Load the `fact_table` DataFrame into a table in the SQLite database
fact_table.to_sql('fact_table', conn, if_exists='replace', index=False)

3574091

In [56]:
#verify loaded data
import pandas as pd
# Read data from a dimension table
dim_query = "SELECT * FROM datetime_dim LIMIT 5;"
dim_df = pd.read_sql_query(dim_query, conn)
display(dim_df.head())

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2025-08-01 00:52:23,0,1,8,2025,4,2025-08-01 01:12:20,1,1,8,2025,4
1,1,2025-08-01 00:03:01,0,1,8,2025,4,2025-08-01 00:15:33,0,1,8,2025,4
2,2,2025-08-01 00:24:38,0,1,8,2025,4,2025-08-01 00:24:38,0,1,8,2025,4
3,3,2025-08-01 00:48:19,0,1,8,2025,4,2025-08-01 00:48:19,0,1,8,2025,4
4,4,2025-08-01 00:25:34,0,1,8,2025,4,2025-08-01 00:33:18,0,1,8,2025,4


In [57]:
# Read data from the fact table
fact_query = "SELECT * FROM fact_table LIMIT 5;"
fact_df = pd.read_sql_query(fact_query, conn)
display(fact_df.head())

,trip id,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,vendor_id,location_id
0,0,0,0,0,0,N,0,33.8,6.0,0.5,5.00,6.94,1.0,57.49,0,0
1,1,1,1,1,1,N,1,21.2,6.0,0.5,0.00,0.00,1.0,30.45,1,1
2,2,2,2,2,2,N,2,14.2,0.0,0.5,3.99,0.00,1.0,23.94,2,2
3,3,3,3,3,3,N,3,11.4,0.0,0.5,3.43,0.00,1.0,20.58,3,3
4,4,4,4,4,4,N,4,11.4,1.0,0.5,2.57,0.00,1.0,19.72,4,4


In [58]:
#Close the connection to the SQLite database
conn = sqlite3.connect('nyc_taxi_data.db')